In [ ]:
import random
import requests
from bs4 import BeautifulSoup
import time
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import csv

In [ ]:
# ip = 'http://185.162.230.48:80'


# def get_proxies(ip):    
#     url = 'https://free-proxy-list.net/anonymous-proxy.html'
#     headers = {"Accept-Language": "en-US,en;q=0.5"}
#     time.sleep(5)
#     response = requests.get(url, headers=headers, proxies={'http': ip})
#     content = BeautifulSoup(response.content, 'html.parser')
#     rows = content.find('table').find_all('tr')
#     rows = rows[1:]
#     proxies = []
#     for row in rows:
#         tds = row.find_all('td')
#         if tds[-2].text == 'no':
#             proxy = f'http://{tds[0].text}:{tds[1].text}'
#         if tds[-2].text == 'yes':
#             proxy = f'https://{tds[0].text}:{tds[1].text}'
#         proxies.append(proxy)
#     return proxies

In [ ]:
with open('proxies.csv', 'r') as f:
    csv_reader = csv.reader(f)
    ips = list(csv_reader)

ip_addresses = [ip[0] for ip in ips]

session = requests.Session()

def proxy_request(url, ip_addresses):
    while True:
        try:
            ua = UserAgent()    
            headers = {"Accept-Language": "en-US,en;q=0.5", 'User-Agent': ua.random}
            proxy = random.randint(0, len(ip_addresses) - 1)
            proxies = {"http": ip_addresses[proxy]}
            response = session.get(url, proxies=proxies, timeout=5, headers=headers)
            if response.status_code == 200:
                break
            elif response.status_code == 429:
                print(f"Rate limit exceeded. Waiting before retrying...")
                time.sleep(10)
            else:
                print(f"Request failed with status code: {response.status_code}")
                time.sleep(random.uniform(1, 3))
        except Exception as e:
            print(f"Error: {e}")

    return response

In [ ]:
url = 'https://www.gsmarena.com/makers.php3'
response = proxy_request(url, ip_addresses)
content = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

In [ ]:
prefix = 'https://www.gsmarena.com/makers.php3/'
maker_urls = dict()
query = list(map(lambda x: x.find('a'), content.find('table').find_all('td')))
maker_urls = dict(zip(list(map(lambda x: x.text.split()[0], query)), list(map(lambda x: prefix + x.attrs['href'], query))))

In [ ]:
maker_urls

In [ ]:
maker_urls['Acer100']

In [ ]:
brand_list = [
    'Samsung1382',
    'Apple118',
    'Xiaomi377',
    'Huawei441',
    'alcatel409',
    'Sony',
    'Sony158',
    'LG667',
    'Lenovo246',
    'ZTE369',
    'Nokia576',
    'BLU369',
    'Infinix125',
    'HTC287',
    'Asus203',
]

In [ ]:
prefix = 'https://www.gsmarena.com/'
maker_brands = dict()

for maker in maker_urls:
    if maker in brand_list:
        maker_brands[maker] = {}  # Initialize the dictionary for the maker
        next_url = maker_urls[maker]  # Start with the first page URL
        while next_url:
            time.sleep(3)  # Respectful crawling by waiting
            response = proxy_request(next_url, ip_addresses)  # Assuming proxy_request function handles the request
            content = BeautifulSoup(response.content, 'html.parser')
            
            # Process the current page
            query = content.find('div', {'class': 'makers'}).find_all('a')
            maker_brands[maker].update(dict(map(lambda x: (x.text, prefix + x.attrs['href']), query)))
            
            # Check for the next page
            next_page = content.find('div', {'class': 'nav-pages'}).find('a', {'title': 'Next page'})
            if next_page:
                next_url = prefix + next_page.attrs['href']  # Update the URL for the next request
            else:
                next_url = None  # No more pages, end the loop
            
            print(maker, "Page processed")
    else:
        continue

In [ ]:
maker_brands

In [ ]:
import json
file_path = 'urls.json'
with open(file_path, 'w') as json_file:
    json.dump(maker_urls, json_file)

In [ ]:
info_dict_with_headers = {}

for brand in maker_brands:
    if brand in brand_list:
        info_dict_with_headers[brand] = {}
        for device in maker_brands[brand]:
            print(device)
            url = maker_brands[brand][device]
            # url = url.replace('https://www.gsmarena.com/makers.php3', 'https://www.gsmarena.com/makers.php3/')
            time.sleep(3)
            response = proxy_request(url, ip_addresses)
            soup = BeautifulSoup(response.content, 'html.parser')
            table_list = soup.find_all('table')
            info_dict_with_headers[brand][device] = {}

            for table in table_list:
                # Attempt to find the table header
                header = table.find('th')
                if header:
                    header_text = header.get_text(strip=True)
                else:
                    # If no <th> is found, or there is a need to identify the header differently, adjust here
                    header_text = "Unknown"  # Placeholder if no header is found or defined differently

                # Initialize the dictionary for this header if not already present
                if header_text not in info_dict_with_headers:
                    info_dict_with_headers[brand][device][header_text] = []

                last_key = None  # Keep track of the last non-empty key for rows without a 'ttl'

                # Iterate through each row in the table
                for row in table.find_all('tr'):
                    ttl_element = row.find("td", class_="ttl")
                    nfo_element = row.find("td", class_="nfo")

                    if ttl_element and ttl_element.get_text(strip=True):
                        # If ttl_element has text, update last_key
                        key = ttl_element.get_text(strip=True)
                        last_key = key
                    else:
                        # If ttl_element is empty, use the last_key
                        key = last_key

                    if nfo_element:
                        value = nfo_element.get_text(strip=True)
                        # Append the key-value pair under the current header
                        if key:
                            info_dict_with_headers[brand][device][header_text].append({key: value})
                        else:
                            # Handle case where there's a value without a key (could append to last key's value or handle as needed)
                            pass

            print(f'{device} Done!')
        print(f'{brand} Done!')
    else:
        continue

In [ ]:
file_path = 'complete_required_data.json'
with open(file_path, 'w') as json_file:
    json.dump(info_dict_with_headers, json_file)

In [ ]:
info_dict_with_headers

In [ ]:
prefix = 'https://www.gsmarena.com/'
maker_brands = dict()

for maker in maker_urls:
    if maker not in brand_list:
        maker_brands[maker] = {}  # Initialize the dictionary for the maker
        next_url = maker_urls[maker]  # Start with the first page URL
        while next_url:
            time.sleep(3)  # Respectful crawling by waiting
            response = proxy_request(next_url, ip_addresses)  # Assuming proxy_request function handles the request
            content = BeautifulSoup(response.content, 'html.parser')
            
            # Process the current page
            query = content.find('div', {'class': 'makers'}).find_all('a')
            maker_brands[maker].update(dict(map(lambda x: (x.text, prefix + x.attrs['href']), query)))
            
            # Check for the next page
            next_page = content.find('div', {'class': 'nav-pages'}).find('a', {'title': 'Next page'})
            if next_page:
                next_url = prefix + next_page.attrs['href']  # Update the URL for the next request
            else:
                next_url = None  # No more pages, end the loop
            
            print(maker, "Page processed")
    else:
        continue

In [ ]:
info_dict_with_headers = {}

for brand in maker_urls:
    if brand not in brand_list:
        info_dict_with_headers[brand] = {}
        for device in maker_urls[brand]:
            print(device)
            url = maker_urls[brand][device]
            url = url.replace('https://www.gsmarena.com/makers.php3', 'https://www.gsmarena.com/makers.php3/')
            time.sleep(3)
            response = proxy_request(url, ip_addresses)
            soup = BeautifulSoup(response.content, 'html.parser')
            table_list = soup.find_all('table')
            info_dict_with_headers[brand][device] = {}

            for table in table_list:
                # Attempt to find the table header
                header = table.find('th')
                if header:
                    header_text = header.get_text(strip=True)
                else:
                    # If no <th> is found, or there is a need to identify the header differently, adjust here
                    header_text = "Unknown"  # Placeholder if no header is found or defined differently

                # Initialize the dictionary for this header if not already present
                if header_text not in info_dict_with_headers:
                    info_dict_with_headers[brand][device][header_text] = []

                last_key = None  # Keep track of the last non-empty key for rows without a 'ttl'

                # Iterate through each row in the table
                for row in table.find_all('tr'):
                    ttl_element = row.find("td", class_="ttl")
                    nfo_element = row.find("td", class_="nfo")

                    if ttl_element and ttl_element.get_text(strip=True):
                        # If ttl_element has text, update last_key
                        key = ttl_element.get_text(strip=True)
                        last_key = key
                    else:
                        # If ttl_element is empty, use the last_key
                        key = last_key

                    if nfo_element:
                        value = nfo_element.get_text(strip=True)
                        # Append the key-value pair under the current header
                        if key:
                            info_dict_with_headers[brand][device][header_text].append({key: value})
                        else:
                            # Handle case where there's a value without a key (could append to last key's value or handle as needed)
                            pass

            print(f'{device} Done!')
        print(f'{brand} Done!')
    else:
        continue

In [ ]:
import json

In [ ]:
file_path = 'complete_data.json'
with open(file_path, 'w') as json_file:
    json.dump(info_dict_with_headers, json_file)